# Analyzing the Surge of EVs in Washington State

In [1]:
# Importing libraries
import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import plotly.graph_objects as go
import plotly.express as px
import re
import folium
from folium import IFrame
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
alt.data_transformers.enable("vegafusion")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('vegafusion')

## Data Inspection and Cleaning

In [2]:
# Reading csv file into df
df = pd.read_csv("../data/Electric_Vehicle_Population_Data.csv")

In [3]:
# Printing first 5 rows of df
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,3C3CFFGE4E,Yakima,Yakima,WA,98902.0,2014,FIAT,500,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,87,0,14.0,1593721,POINT (-120.524012 46.5973939),PACIFICORP,5.307700e+10
1,5YJXCBE40H,Thurston,Olympia,WA,98513.0,2017,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,200,0,2.0,257167501,POINT (-122.817545 46.98876),PUGET SOUND ENERGY INC,5.306701e+10
2,3MW39FS03P,King,Renton,WA,98058.0,2023,BMW,330E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,20,0,11.0,224071816,POINT (-122.1298876 47.4451257),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
3,7PDSGABA8P,Snohomish,Bothell,WA,98012.0,2023,RIVIAN,R1S,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,21.0,260084653,POINT (-122.1873 47.820245),PUGET SOUND ENERGY INC,5.306105e+10
4,5YJ3E1EB8L,King,Kent,WA,98031.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,322,0,33.0,253771913,POINT (-122.2012521 47.3931814),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10


In [4]:
# Inspecting for nulls and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166800 entries, 0 to 166799
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         166800 non-null  object 
 1   County                                             166795 non-null  object 
 2   City                                               166795 non-null  object 
 3   State                                              166800 non-null  object 
 4   Postal Code                                        166795 non-null  float64
 5   Model Year                                         166800 non-null  int64  
 6   Make                                               166800 non-null  object 
 7   Model                                              166800 non-null  object 
 8   Electric Vehicle Type                              166800 non-null  object

In [5]:
# Filtering the df to include only rows where the 'State' column is 'WA'
df = df[df['State'] == 'WA']

## Visualization 1 - Map Chart

In [6]:
# Function to extract coordinate data
def extract_coordinates(point_str):
    """
    Extracts latitude and longitude from a string representing a point.

    Params:
    point_str (str): A string representing a point in the format 'POINT (longitude latitude)'.

    Returns:
    tuple: A tuple containing two float values representing latitude and longitude. 
           If the input is not valid, it returns (None, None).
    """
    if isinstance(point_str, str):
        # Removing 'POINT (' and ')' and splitting by space
        coords = point_str.replace('POINT (', '').replace(')', '').split()
        return float(coords[1]), float(coords[0])  # Note: coords are in order of latitude, longitude
    return None, None

# Applying extraction function and dropping nulls
df['Latitude'], df['Longitude'] = zip(*df['Vehicle Location'].apply(extract_coordinates))
df = df.dropna(subset=['Latitude', 'Longitude'])

# Grouping by location to find unique points
grouped = df.groupby(['Latitude', 'Longitude'])

# Creating a new Folium map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)

# Adding html title
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Geographic Distribution of EV Registrations in WA</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Creating a MarkerCluster object
marker_cluster = MarkerCluster().add_to(m)

# Adding summarized markers to the cluster
for (lat, lon), group in grouped:
    # Getting the most common county name in the group
    county_name = group['County'].mode()[0]

    # Getting the top 5 makes and models
    top_makes_models = group.groupby(['Make', 'Model']).size().nlargest(5).reset_index(name='Count')

    # Making a summary string for the popup
    summary_str = "<br>".join([f"<b>{row['Make']} {row['Model']}:</b> {row['Count']}" for index, row in top_makes_models.iterrows()])
    
    # Making the content for the popup
    popup_content = f"""<div style="width:300px; max-height:200px; overflow:auto;">
                            <b>County:</b> {county_name}<br>
                            <b>Total cars:</b> {len(group)}<br><br>
                            <b>Top Makes and Models:</b><br><br>
                            {summary_str}
                        </div>
                    """
    
    # Creating and adding the marker to the cluster
    folium.Marker(
        location=[lat, lon],
        popup=popup_content
    ).add_to(marker_cluster)

# Outputting chart
m

In [7]:
# Saving the map chart to an HTML file
m.save('map_chart.html')

## Visualization 2 - Line Chart

In [8]:
# Converting 'Model Year' to a string 
df['Model Year'] = df['Model Year'].astype(str)

# Aggregating the data to count registrations per year per EV type
agg_data = df.groupby(['Model Year', 'Electric Vehicle Type']).size().reset_index(name='Registrations')

# Setting dropdown options
ev_types = ['Battery Electric Vehicle (BEV)', 'Plug-in Hybrid Electric Vehicle (PHEV)']

# Setting a dropdown filter
ev_dropdown = alt.binding_select(options=ev_types, name="EV Types")
ev_select = alt.selection_point(fields=['Electric Vehicle Type'], bind=ev_dropdown)

# Creating the line chart
base = alt.Chart(agg_data).mark_line(point=True).encode(
    x=alt.X('Model Year:O', axis=alt.Axis(title='Model Year')),
    y=alt.Y('Registrations:Q', axis=alt.Axis(title='Number of Registrations')),
    color='Electric Vehicle Type:N',
    tooltip=['Model Year', 'Electric Vehicle Type', 'Registrations']
).interactive().properties(
    title='Trend of Electric Vehicle Registrations by Type',
    width=400,
    height=300
)

# Adding dropdown
line_chart = base.add_params(
    ev_select
).transform_filter(
    ev_select
)

# Outputting chart
line_chart

alt.Chart(...)

In [9]:
# Saving the line chart to an HTML file
line_chart.save('line_chart.html')

## Visualization 3 - Vertical Bar Charts

In [10]:
# Grouping and counting the occurrences of each electric vehicle type within each make
grouped_df = df.groupby(['Make', 'Electric Vehicle Type']).size().reset_index(name='Count')

# Filtering to include only the relevant electric vehicle types
relevant_types = ['Battery Electric Vehicle (BEV)', 'Plug-in Hybrid Electric Vehicle (PHEV)']
grouped_df = grouped_df[grouped_df['Electric Vehicle Type'].isin(relevant_types)]

# Finding the top makes by the sum of counts for both BEV and PHEV
top_makes_df = grouped_df.groupby('Make')['Count'].sum().nlargest(10).reset_index()

# Filtering the original grouped_df to only include the top makes
grouped_df = grouped_df[grouped_df['Make'].isin(top_makes_df['Make'])]

# Setting a dropdown filter
ev_dropdown = alt.binding_select(options=ev_types, name="EV Types")
ev_select = alt.selection_point(fields=['Electric Vehicle Type'], bind=ev_dropdown)

# Creating the side-by-side vertical bar charts with log scale
base = alt.Chart(grouped_df).mark_bar().encode(
    x=alt.X('Make:N', title='Make'),
    y=alt.Y('Count:Q', title='Number of Vehicles (log scale)', scale=alt.Scale(type='log')),
    color=alt.Color('Electric Vehicle Type:N', title='Electric Vehicle Type'),
    column=alt.Column('Electric Vehicle Type:N', header=alt.Header(title=None)),  
    tooltip=[alt.Tooltip('Make:N', title='Make'),
             alt.Tooltip('Electric Vehicle Type:N', title='Vehicle Type'),
             alt.Tooltip('Count:Q', title='Number of Vehicles')]
).properties(
    width=300,
    height=400,
    title='Comparison of Electric Vehicle Types by Make'
).configure_title(
    align='center',
    anchor='middle'  
).properties(
    title='Comparison of Electric Vehicle Types by Make (Log Scale)'
)

# Adding dropdown
side_by_side_bar_chart = base.add_params(
    ev_select
).transform_filter(
    ev_select
)

# Outputting charts
side_by_side_bar_chart

alt.Chart(...)

In [11]:
# Saving the vertical bar charts to an HTML file
side_by_side_bar_chart.save('vertical_bar_charts.html')

## Visualization 4 - Bar Chart

In [12]:
# Getting count of utility firms
counts = df['Electric Utility'].value_counts().reset_index()

# Renaming columns and sorting in descending order
counts.columns = ['Electric Utility Firm', 'Count']  
counts = counts.sort_values(by='Count', ascending=False).head(10)  

# Creating a horizontal bar chart without tooltip
horizontal_bar_chart = alt.Chart(counts).mark_bar().encode(
    y=alt.Y('Electric Utility Firm:N', sort='-x'), 
    x=alt.X('Count:Q'),
    color=alt.Color('Count:Q', legend=None)
).properties(
    width=800,
    height=450,
    title='Top 10 Electric Utility Firms Used in WA'
)

# Outputting chart
horizontal_bar_chart

alt.Chart(...)

In [13]:
# Saving the horizontal bar chart to an HTML file
horizontal_bar_chart.save('horizontal_bar_chart.png')